<a href="https://colab.research.google.com/github/elMimu/pln-project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

`Pedro Ives Silva Santos - RA: 11201920234`

**Integrante 02:**

`Michel Munhoz - RA: 11017916`

### **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---

>


**LLM**: Google Gemini 2.5 Flash

>

**Link para a documentação oficial**: https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-flash



### **API (Opcional)**
---

**API**: TMDB API

**Site oficial**: https://www.themoviedb.org/

**Link para a documentação oficial**: https://developer.themoviedb.org/docs/getting-started

### **IMPLEMENTAÇÃO**
---

# 📦 Instalação de Dependências e Importações

Antes de iniciarmos o projeto, precisamos preparar o ambiente do Google Colab.

**O que esta célula faz:**
1.  **Instalação (`pip install`):** Baixa as versões mais recentes do `langchain`, `langchain-google-genai` e utilitários.
2.  **Importações:** Carrega os módulos necessários para conexão com APIs, manipulação de variáveis de ambiente e criação dos templates de prompt.

In [ ]:
# Instalação de dependências
!pip install -q langchain langchain-google-genai google-genai python-dotenv requests

In [ ]:
# --- IMPORTS ---
import os
import requests
from getpass import getpass

# Utilitários do Google Colab e Ambiente
from google.colab import drive
from dotenv import load_dotenv

# Componentes do LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

print("✅ Dependências instaladas e bibliotecas importadas com sucesso!")

✅ Dependências instaladas e bibliotecas importadas com sucesso!


# 🔒 Configuração Segura de Ambiente

Esta etapa realiza a autenticação segura do projeto utilizando o recurso nativo **Google Colab Secrets**.

**Como funciona:**
1.  **Recuperação Segura:** O script acessa o cofre de credenciais do Colab (ícone 🔑) para buscar as chaves `GOOGLE_API_KEY` e `TMDB_API_KEY`.
2.  **Variáveis de Ambiente:** As chaves são injetadas diretamente na memória (`os.environ`), simulando o comportamento de um arquivo `.env`.
3.  **Segurança:** Nenhuma chave é exposta no código ou salva em arquivos de texto, garantindo que o notebook possa ser compartilhado sem riscos de vazamento de credenciais.

In [ ]:
from google.colab import userdata

def load_colab_secrets():
    try:
        # Recupera as chaves do cofre do Colab
        google_key = userdata.get('GOOGLE_API_KEY')
        tmdb_key = userdata.get('TMDB_API_KEY')

        # Define como variáveis de ambiente
        os.environ["GOOGLE_API_KEY"] = google_key
        os.environ["TMDB_API_KEY"] = tmdb_key

        print("✔ Chaves carregadas com sucesso via Colab Secrets!")
        print(f"✔ GOOGLE_API_KEY definida? {bool(os.getenv('GOOGLE_API_KEY'))}")
        print(f"✔ TMDB_API_KEY definida? {bool(os.getenv('TMDB_API_KEY'))}")

    except userdata.SecretNotFoundError as e:
        print("⚠ Erro: Uma das chaves não foi encontrada nos Segredos.")
        print("Certifique-se de adicionar 'GOOGLE_API_KEY' e 'TMDB_API_KEY' no ícone de chave (🔑) na barra lateral.")
    except Exception as e:
        print(f"⚠ Erro inesperado: {e}")

# Executa a configuração
load_colab_secrets()

✔ Chaves carregadas com sucesso via Colab Secrets!
✔ GOOGLE_API_KEY definida? True
✔ TMDB_API_KEY definida? True


# 🤖 Inicialização do Modelo LLM (Gemini)

Nesta etapa, instanciamos o cliente da Inteligência Artificial que será o "cérebro" da aplicação.

**Detalhes da Configuração:**
* **Modelo (`gemini-2.5-flash`):** Escolhido por ser rápido e eficiente, ideal para tarefas que exigem baixa latência.
* **Temperatura (`0.3`):** Define a "criatividade" do modelo.
    * Um valor baixo (0.3) torna as respostas mais determinísticas, focadas e diretas.
    * Valores altos (perto de 1.0) tornam o modelo mais criativo e variado.
* **Teste de Sanidade:** Ao final, fazemos uma pergunta simples ("invoke") apenas para garantir que a conexão com a API do Google está ativa e respondendo corretamente.

In [ ]:
# Criando o modelo do Gemini
llm = ChatGoogleGenerativeAI(
    # é possível trocar pro gemini-2.5-pro, porém o rate limit será menor
    model="gemini-2.5-flash",
    temperature=0.3,
)

print("🤖 Modelo Gemini inicializado com sucesso!")

# Teste Rápido de Conexão
try:
    print("⏳ Enviando pergunta de teste...")
    resposta_teste = llm.invoke("Explique em uma frase o que é o TMDB.")

    print("\n--- Resposta do Gemini ---")
    print(resposta_teste.content)
    print("--------------------------")

except Exception as e:
    print(f"❌ Erro ao conectar com o Gemini: {e}")

🤖 Modelo Gemini inicializado com sucesso!
⏳ Enviando pergunta de teste...

--- Resposta do Gemini ---
O TMDB é um banco de dados que oferece informações detalhadas sobre filmes, séries de TV, elenco e equipe.
--------------------------


# 📡 Cliente HTTP para a API do TMDB

Esta célula define uma função auxiliar (`wrapper`) chamada `tmdb_get`. O objetivo dela é simplificar todas as chamadas futuras que faremos ao banco de dados de filmes.

**Funcionalidades:**
* **Autenticação Automática:** Insere a `api_key` em todas as requisições, para que você não precise repeti-la.
* **Idioma Padrão:** Define `pt-BR` (Português) como padrão, mas permite que você sobrescreva se quiser dados em inglês.
* **Tratamento de Erros:** Utiliza `raise_for_status()` para avisar imediatamente caso a API retorne um erro (como 404 ou 401).
* **Formato JSON:** Já retorna os dados prontos para uso em Python (dicionários/listas).

In [ ]:
TMDB_BASE_URL = "https://api.themoviedb.org/3"

def tmdb_get(endpoint, params=None):
    if params is None:
        params = {}

    # Recupera a chave do ambiente (configurada na etapa anterior)
    params["api_key"] = os.environ.get("TMDB_API_KEY")

    # Define o idioma padrão como Português do Brasil, se não informado
    params.setdefault("language", "pt-BR")

    # Monta a URL completa
    url = f"{TMDB_BASE_URL}{endpoint}"

    # Faz a requisição
    response = requests.get(url, params=params)

    # Verifica se houve erro (4xx ou 5xx) e retorna o JSON
    response.raise_for_status()
    return response.json()

print("✔ Função 'tmdb_get' definida e pronta para uso!")

✔ Função 'tmdb_get' definida e pronta para uso!


# 🔎 Função de Busca de Filmes

Esta função abstrai a pesquisa no catálogo do TMDB.

**Como funciona:**
* **Parâmetros:** Recebe o nome do filme (`query`), número da página e idioma.
* **Retorno:** Devolve uma lista de dicionários, onde cada item é um filme encontrado.
* **Filtro:** A função já descarta os metadados da API (como total de páginas) e retorna direto a lista `['results']`.

In [ ]:
def search_movie(query, page=1, language="pt-BR"):
    # Usa a função auxiliar tmdb_get definida anteriormente
    data = tmdb_get("/search/movie", params={
        "query": query,
        "page": page,
        "language": language
    })
    return data["results"]

# --- Teste da Função ---
filme = input()
print(f"🔎 Buscando por '{filme}'...\n")

filmes = search_movie(filme, language="en-US")

# Exibe os 5 primeiros resultados de forma formatada
if filmes:
    for f in filmes[:5]:
        title = f.get('title', 'Sem título')
        date = f.get('release_date', 'Data desconhecida')
        rating = f.get('vote_average', 0)
        print(f"🎬 {title} ({date}) - Nota: {rating}")
else:
    print("❌ Nenhum filme encontrado.")

Batman: The Killing Joke
🔎 Buscando por 'Batman: The Killing Joke'...

🎬 Batman: The Killing Joke (2016-07-24) - Nota: 6.614
🎬 AFSOS's "Batman: The Killing Joke" () - Nota: 0.0


# 📝 Obtenção de Detalhes e Avaliações

A partir da escolha do filme, é necessária a extração de informações profundas para alimentar o modelo.

**As Funções:**
1.  **`get_movie_details`:** A busca retorna apenas o básico. Esta função vai buscar a ficha técnica completa (orçamento, gêneros, duração, status).
2.  **`get_movie_reviews`:** Busca opiniões de usuários reais.
    * *Estratégia:* No exemplo abaixo, optamos por buscar reviews em **Inglês (`en-US`)**, pois a grande maioria das críticas textuais no TMDB não está em português. O Gemini pode traduzi-las ou analisá-las facilmente depois.

In [ ]:
def get_movie_details(movie_id, language="pt-BR"):
    data = tmdb_get(f"/movie/{movie_id}", params={"language": language})
    return data

def get_movie_reviews(movie_id, page=1, language="en-US"):
    data = tmdb_get(f"/movie/{movie_id}/reviews", params={
        "page": page,
        "language": language
    })
    return data["results"]

# --- Teste das Funções ---

# Verifica se a variável 'filmes' da célula anterior existe e tem conteúdo
if 'filmes' in globals() and filmes:
    filme_escolhido = filmes[0] # Pega o primeiro filme da lista
    print(f"🎯 Filme Selecionado: {filme_escolhido['title']} (ID: {filme_escolhido['id']})\n")

    # Pega detalhes em PT-BR
    detalhes = get_movie_details(filme_escolhido["id"], language="pt-BR")

    # Pega reviews em Inglês (maior volume de dados)
    reviews = get_movie_reviews(filme_escolhido["id"], language="en-US")

    print(f"📜 Título Oficial: {detalhes['title']}")
    print(f"⏱  Duração: {detalhes.get('runtime', '?')} minutos")
    print(f"📝 Sinopse: {detalhes.get('overview', 'Sem sinopse.')}\n")

    print(f"💬 Total de reviews encontradas: {len(reviews)}")
    if reviews:
        autor = reviews[0].get('author', 'Anônimo')
        trecho = reviews[0].get('content', '')[:100]
        print(f"   Exemplo ({autor}): \"{trecho}...\"")

else:
    print("⚠ A lista 'filmes' está vazia ou não foi definida.")
    print("Por favor, rode a célula de busca (search_movie) anterior primeiro.")

🎯 Filme Selecionado: Batman: The Killing Joke (ID: 382322)

📜 Título Oficial: Batman: A Piada Mortal
⏱  Duração: 72 minutos
📝 Sinopse: Do produtor executivo Bruce Timm e baseado na aclamada graphic novel da DC Comics, “Batman: A Piada Mortal” é uma jornada adentro da sombria psique do Príncipe Palhaço do Crime. Acompanhando desde o seu humilde começo, quando era um comediante esforçado, até seu fatídico encontro com o Cavaleiro das Trevas, que mudou tudo. Agora fugitivo do Asilo Arkham, o Coringa planeja provar para todos que um dia ruim é capaz de tornar qualquer um tão insano quanto ele.

💬 Total de reviews encontradas: 5
   Exemplo (Austin Singleton): "Not even the voice acting could save this movie. Check out my full review here.

http://www.hweird..."


# 🧠 Análise de Sentimento com LangChain

Selecionadas as críticas (reviews), utilizaremos a Inteligência Artificial para ler e classificar a opinião das pessoas.

**Conceitos Técnicos:**
* **PromptTemplate:** O "molde" que instrui a IA sobre como se comportar (neste caso, um classificador especializado em cinema).
* **JsonOutputParser:** Força a IA a responder *exatamente* no formato JSON. Isso é fundamental para transformar texto livre em dados estruturados (tabelas).
* **LCEL (LangChain Expression Language):** A sintaxe `prompt | llm | parser` cria um "tubo" onde os dados entram de um lado e saem estruturados do outro.

In [ ]:
import pandas as pd

# Configuração do Parser
sentiment_parser = JsonOutputParser()

# Prompt Template"
prompt_template_str = """
Você é um classificador de sentimentos de alta precisão especializado em cinema.
Sua tarefa é ler a crítica abaixo e extrair dados estruturados.

Siga estritamente estes passos:
1. Identifique as palavras-chave de emoção (adjetivos, verbos).
2. Verifique se há sarcasmo (ex: elogios exagerados para filmes ruins).
3. Pese os pontos positivos vs. negativos.
4. Classifique seguindo estas regras RÍGIDAS:
   - "positivo": O autor recomenda o filme ou elogia a maioria dos aspectos.
   - "negativo": O autor não recomenda ou critica a maioria dos aspectos.
   - "neutro": A crítica é informativa, ou os pontos bons e ruins se anulam.

Crítica para análise:
'''
{review_text}
'''

Retorne APENAS um JSON com o seguinte esquema (sem markdown, sem texto extra):
{{
  "key_points": "Liste 2 ou 3 pontos principais citados (ex: Atuação ruim, Ótima fotografia)",
  "sentiment": "positivo, negativo ou neutro",
  "confidence": "Alta, Média ou Baixa",
  "explanation": "Uma frase curta justificando a escolha baseada nos pontos levantados."
}}
"""

sentiment_prompt = ChatPromptTemplate.from_template(prompt_template_str)

# Recriando a Chain
sentiment_chain = sentiment_prompt | llm | sentiment_parser

# --- Execução Otimizada ---

analise_sentimentos = []
# Rate Limit
reviews_para_analisar = reviews[:10] if 'reviews' in globals() else []

print(f"🔬 Iniciando análise de precisão em {len(reviews_para_analisar)} críticas...\n")

for i, r in enumerate(reviews_para_analisar):
    try:
        # Dica visual para saber que está rodando
        print(f"   [{i+1}/{len(reviews_para_analisar)}] Analisando opinião de '{r.get('author', 'Anônimo')}'...")

        result = sentiment_chain.invoke({"review_text": r["content"]})

        analise_sentimentos.append({
            "author": r["author"],
            "sentiment": result.get("sentiment"),
            "confidence": result.get("confidence"), # Novo campo útil
            "key_points": result.get("key_points"),
            "explanation": result.get("explanation"),
            "original_text": r["content"][:100] + "..." # Limite para a review
        })

    except Exception as e:
        print(f"   ❌ Erro ao processar crítica {i+1}: {e}")

# Exibição dos Resultados
if analise_sentimentos:
    df_sentimentos = pd.DataFrame(analise_sentimentos)

    # Função de estilização para destacar resultados
    def style_table(val):
        color = 'black'
        if val == 'positivo': color = 'green'
        elif val == 'negativo': color = 'red'
        elif val == 'neutro': color = 'orange'
        return f'color: {color}; font-weight: bold'

    print("\n✅ Tabela de Sentimentos Gerada:")
    # Aplicando estilo na coluna 'sentiment'
    display(df_sentimentos.style.applymap(style_table, subset=['sentiment']))
else:
    print("\n⚠ Nenhuma crítica disponível para analisar.")

🔬 Iniciando análise de precisão em 5 críticas...

   [1/5] Analisando opinião de 'Austin Singleton'...
   [2/5] Analisando opinião de 'Gimly'...
   [3/5] Analisando opinião de 'Peter89Spencer'...
   [4/5] Analisando opinião de 'GenerationofSwine'...
   [5/5] Analisando opinião de 'Murp'...

✅ Tabela de Sentimentos Gerada:


/tmp/ipython-input-3271775458.py:80: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(df_sentimentos.style.applymap(style_table, subset=['sentiment']))


,author,sentiment,confidence,key_points,explanation,original_text
0,Austin Singleton,negativo,Alta,"Filme ruim, Atuação de voz não foi suficiente para salvar o filme","A frase indica que o filme é ruim, pois nem mesmo um aspecto potencialmente positivo (atuação de voz) foi capaz de salvá-lo.",Not even the voice acting could save this movie. Check out my full review here. http://www.hweird...
1,Gimly,negativo,Alta,"Falta de coesão devido à adição forçada de uma nova história, falha em atender às altas expectativas apesar da boa atuação de voz de Hamill, e o filme 'não funcionou bem como um todo'.","A crítica aponta diversas fraquezas, falta de coesão e decepção geral, superando os poucos pontos positivos específicos.","Shares the few weaknesses of the short comic it is based on, but adds a bunch of its own new weaknes..."
2,Peter89Spencer,negativo,Alta,"Primeiros 30 minutos desperdiçados e irritantes, A atuação de Mark Hamill não salvou o filme, História poderia ter sido melhor.","A crítica aponta múltiplos aspectos negativos, como o início desperdiçado e a história fraca, com apenas um ponto positivo mínimo para uma piada.",The first 30 minutes was wasted with Barbara being so annoying! I just didn't see why that whole bit...
3,GenerationofSwine,positivo,Alta,"A adaptação diverge do material original, mas é considerada melhor que a maioria dos outros filmes do Batman e consegue contar uma boa história.","Apesar da decepção inicial com a fidelidade à graphic novel, a crítica elogia o filme por ser superior a outros filmes do Batman e por sua narrativa eficaz.","OK, so I'll admit I was expecting something closer to the graphic novel, and I was pretty disappoint..."
4,Murp,neutro,Alta,"Primeira metade é irrelevante e filler, A adaptação principal (segunda metade) é ótima com atuações clássicas, O filme é uma mistura de aspectos amados e odiados","A crítica explicitamente divide o filme em partes amadas e odiadas ('Half of this movie I despise and half of this I love'), com a nota 5/10 reforçando o equilíbrio.",_The Killing Joke_ deserved a better adaptation. Not even that. Just re-edit this to cut out the fir...


# 🏷️ Extração Híbrida: Tópicos e Elenco

Ajustamos o modelo para realizar uma tarefa dupla. Agora ele busca tanto a **opinião** quanto os **sujeitos** da crítica.

**O que ele vai extrair agora:**
1.  **Sentimentos (Bigramas):** "Atuação Incrível", "Ritmo Lento".
2.  **Entidades (Nomes):** Nomes de atores, diretor e personagens principais citados na crítica.

**Regra de Ouro:**
* Conceitos são traduzidos para PT-BR.
* Nomes próprios (Atores/Personagens) são mantidos no original para evitar confusão.

In [ ]:
import pandas as pd
from collections import Counter

# --- CONFIGURAÇÃO DO PROMPT HÍBRIDO ---
keywords_parser = JsonOutputParser()

keywords_prompt = ChatPromptTemplate.from_template("""
Você é um analista de metadados de cinema.
Analise a crítica e extraia uma lista mista de tags relevantes.

Siga estas 3 regras de extração:
1. **SENTIMENTOS (Bigramas):** Identifique pontos fortes/fracos combinando Aspecto + Adjetivo (ex: "Roteiro Confuso", "Visual Sombrio").
2. **ENTIDADES (Nomes):** Identifique os principais Atores, Diretor ou Personagens citados.
3. **IDIOMA:** - Conceitos gerais: Traduza para Português.
   - Nomes Próprios: MANTENHA O ORIGINAL (ex: "Joker", "Phoenix", "Gaga").

Crítica:
'''{review_text}'''

Retorne JSON:
{{
  "keywords": ["tag1", "tag2", "tag3", "tag4", "tag5"]
}}
""")

keywords_chain = keywords_prompt | llm | keywords_parser

# --- EXECUÇÃO ---

analise_keywords = []
lista_reviews = reviews[:10] if 'reviews' in globals() else []
total = len(lista_reviews)

print(f"🚀 Extraindo Tópicos de {total} críticas...\n")

for i, r in enumerate(lista_reviews):
    try:
        print(f"   [{i+1}/{total}] Analisando '{r.get('author', 'Anônimo')}'...")

        result = keywords_chain.invoke({"review_text": r["content"]})
        tags_brutas = result.get("keywords", [])

        # Cleaning
        tags_limpas = [tag.strip().title() for tag in tags_brutas]

        analise_keywords.append({
            "author": r["author"],
            "keywords": tags_limpas
        })

    except Exception as e:
        print(f"   ❌ Erro na crítica {i+1}: {e}")

# --- ANÁLISE DE RESULTADOS ---

print("\n✅ Extração Híbrida Concluída!")

if analise_keywords:
    df_keywords = pd.DataFrame(analise_keywords)

    todas_tags = [k for lista in df_keywords['keywords'] for k in lista]
    contagem = Counter(todas_tags)

    print(f"\n📊 Total de tags: {len(todas_tags)}")
    print("🏆 Top 10 Tags:")

    for tag, qtd in contagem.most_common(15):
        tipo = "📝" if " " in tag else "👤"
        print(f"   {tipo} {tag}: {qtd}")
else:
    print("⚠ Nenhuma tag extraída.")

🚀 Extraindo Tópicos de 5 críticas...

   [1/5] Analisando 'Austin Singleton'...
   [2/5] Analisando 'Gimly'...
   [3/5] Analisando 'Peter89Spencer'...
   [4/5] Analisando 'GenerationofSwine'...
   [5/5] Analisando 'Murp'...

✅ Extração Híbrida Concluída!

📊 Total de tags: 41
🏆 Top 10 Tags:
   👤 Joker: 4
   📝 Mark Hamill: 3
   📝 The Killing Joke: 2
   📝 Kevin Conroy: 2
   👤 Barbara: 2
   👤 Batman: 2
   📝 Atuação De Voz Insuficiente: 1
   📝 Recepção Negativa: 1
   📝 Expectativas Não Atendidas: 1
   📝 Filme Não Coeso: 1
   📝 História Forçada: 1
   📝 Conteúdo Grotesco: 1
   📝 Cena Final Fantástica: 1
   📝 Hamill Ótimo: 1
   📝 Performance Vocal Imperfeita: 1


# 📢 O Veredito (Sumarização)

**O Processo:**
1.  **Agregação:** Juntamos todos os textos das críticas em uma única string gigante.
2.  **Síntese:** Pedimos para a IA ignorar ruídos e focar no consenso: o que a maioria amou e o que a maioria odiou?

In [ ]:
from IPython.display import Markdown, display

# Preparação dos Dados
# Verifica se temos reviews carregadas
if 'reviews' in globals() and reviews:

    # 10 primeiras reviews como amostra
    amostra_reviews = reviews[:10]

    all_reviews_text = "\n\n--- NOVA CRÍTICA ---\n\n".join([r["content"] for r in amostra_reviews])

    print(f"📚 Lendo e sintetizando {len(amostra_reviews)} opiniões dos usuários...\n")

    # Configuração do Prompt
    summary_prompt = ChatPromptTemplate.from_template("""
    Você é um editor sênior de um portal de cinema.
    Sua tarefa é ler as opiniões do público abaixo e criar um RESUMO EXECUTIVO.

    Instruções:
    1. Ignore opiniões extremas isoladas (outliers). Foque no consenso.
    2. Identifique padrões: O que a maioria elogia? O que a maioria critica?
    3. Escreva em Português do Brasil, de forma fluida e jornalística.
    4. Não faça listas de tópicos, escreva 2 ou 3 parágrafos bem conectados.

    Críticas dos usuários:
    '''{all_reviews}'''
    """)

    summary_chain = summary_prompt | llm

    # Execução
    try:
        resumo_opinioes = summary_chain.invoke({"all_reviews": all_reviews_text})

        # Exibição Formatada
        print("✅ Resumo Gerado:\n")
        display(Markdown(resumo_opinioes.content))

    except Exception as e:
        print(f"❌ Erro ao gerar resumo: {e}")

else:
    print("⚠ Nenhuma crítica encontrada para resumir. Rode as etapas anteriores.")

📚 Lendo e sintetizando 5 opiniões dos usuários...

✅ Resumo Gerado:



A aguardada adaptação de "A Piada Mortal" para o cinema gerou expectativas altíssimas entre o público, impulsionadas pelo retorno de Kevin Conroy e Mark Hamill aos seus icônicos papéis e pela reverência à aclamada graphic novel. Contudo, a recepção geral foi de grande decepção, com a maioria dos espectadores apontando a falta de coesão e a inserção de material extra como os principais problemas. O primeiro ato do filme, focado em Barbara Gordon, é unanimemente criticado como desnecessário, mal-executado e um "enchimento" que não se conecta com a trama principal, prejudicando a fluidez e a fidelidade à obra original. Muitos sentiram que essa porção inicial transformou o filme em duas histórias distintas, em vez de uma narrativa única e bem integrada.

Apesar das fortes críticas à estrutura, há um consenso quase absoluto sobre a excelência das performances vocais, especialmente a de Mark Hamill como Coringa. Sua interpretação é consistentemente elogiada como icônica e um dos grandes pontos altos da produção. Kevin Conroy também é reconhecido por entregar uma performance clássica como Batman, embora alguns poucos tenham notado pequenas inconsistências. A parte do filme que se dedica à adaptação direta da graphic novel original, após o controverso primeiro ato, é geralmente vista como bem-sucedida, com muitos a descrevendo como "fantástica" e o ponto onde a adaptação realmente brilha.

Em resumo, "A Piada Mortal" é percebida como uma obra de duas metades: uma primeira frustrante e desconexa que falha em justificar sua existência, e uma segunda que, ao se ater ao material-fonte, entrega momentos de brilho e as atuações esperadas dos lendários dubladores. A adaptação, no entanto, falha em ser um todo coeso e em corresponder plenamente ao legado da aclamada história em quadrinhos, deixando um gosto amargo de oportunidade perdida para muitos fãs.

# 🤖 Chatbot Especialista

Nesta etapa final, consolidamos todo o conhecimento gerado em um **Assistente Virtual Interativo**.

**Como funciona (Context Injection):**
1.  **Construção do Contexto:** O código reúne os dados técnicos, as estatísticas de sentimento, as palavras-chave mais citadas e o resumo textual que geramos anteriormente.
2.  **O Prompt do Sistema:** Dizemos à IA: *"Responda às perguntas do usuário baseando-se EXCLUSIVAMENTE nestes dados que coletei."*
3.  **Interação:** Você pode fazer perguntas complexas como *"O filme é muito longo?"* ou *"Vale a pena ver se eu gosto de ação?"* e ele responderá usando as evidências das críticas reais.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from IPython.display import display, Markdown

# --- Criação do Contexto ---

contexto_dados = f"""
DADOS TÉCNICOS DO FILME:
- Título: {detalhes.get('title')}
- Nota Média (TMDB): {detalhes.get('vote_average')}
- Duração: {detalhes.get('runtime')} minutos
- Sinopse: {detalhes.get('overview')}

ESTATÍSTICAS DAS CRÍTICAS PROCESSADAS:
"""

# Adiciona dados de sentimento se existirem
if 'df_sentimentos' in globals() and not df_sentimentos.empty:
    pos = len(df_sentimentos[df_sentimentos['sentiment'] == 'positivo'])
    total = len(df_sentimentos)
    contexto_dados += f"- Aprovação nesta amostra: {pos}/{total} críticas positivas.\n"

# Adiciona Top Keywords se existirem
if 'contagem' in globals() and contagem:
    top_tags = [k for k, v in contagem.most_common(10)]
    contexto_dados += f"- Principais Tópicos Comentados: {', '.join(top_tags)}.\n"

# Adiciona o Resumo Textual gerado na etapa anterior
if 'resumo_opinioes' in globals():
    contexto_dados += f"\nRESUMO QUALITATIVO DA AUDIÊNCIA:\n{resumo_opinioes.content}\n"

# --- Configuração do Chatbot ---

chat_prompt = ChatPromptTemplate.from_template("""
Você é um assistente de cinema especializado no filme '{titulo}'.
Sua fonte de verdade é o CONTEXTO abaixo.

CONTEXTO (Dados reais analisados):
{contexto}

Instruções:
1. Responda à pergunta do usuário usando as evidências do contexto.
2. Se a informação não estiver no contexto, diga "Não tenho dados suficientes nas críticas analisadas para afirmar isso".
3. Seja prestativo, curto e direto.

Pergunta do Usuário: {pergunta}
""")

chat_chain = chat_prompt | llm | StrOutputParser()

# --- Loop de Interação ---
print(f"💬 Chatbot iniciado para o filme: {detalhes.get('title')}")
print("Digite 'sair' para encerrar.\n")

while True:
    pergunta_user = input("🤔 Você: ")

    if pergunta_user.lower() in ['sair', 'exit', 'fim']:
        print("👋 Chat encerrado.")
        break

    if pergunta_user.strip() == "":
        continue

    # Resposta
    print("🤖 Gemini pensando...", end="\r")
    resposta = chat_chain.invoke({
        "titulo": detalhes.get('title'),
        "contexto": contexto_dados,
        "pergunta": pergunta_user
    })

    # Formatação
    display(Markdown(f"**🤖 Gemini:** {resposta}"))
    print("-" * 50)

💬 Chatbot iniciado para o filme: Batman: A Piada Mortal
Digite 'sair' para encerrar.

🤔 Você: O que voce diria para mim sobre esse filme?


**🤖 Gemini:** "Batman: A Piada Mortal" é uma adaptação da aclamada graphic novel, com 72 minutos de duração e uma nota média de 6.614 no TMDB. A sinopse descreve uma jornada pela psique do Coringa, desde seu começo como comediante até seu plano de provar que um dia ruim pode levar qualquer um à insanidade.

Apesar das altas expectativas, impulsionadas pelo retorno de Kevin Conroy e Mark Hamill, a recepção geral foi de grande decepção. O principal problema apontado é a falta de coesão e a inserção de material extra, especialmente o primeiro ato focado em Barbara Gordon, que foi amplamente criticado como desnecessário e desconexo.

Por outro lado, as performances vocais são unanimemente elogiadas, com Mark Hamill como Coringa sendo considerado icônico e Kevin Conroy entregando uma performance clássica como Batman. A parte do filme que adapta diretamente a graphic novel original é vista como bem-sucedida e brilhante.

Em resumo, o filme é percebido como uma obra de duas metades: uma primeira frustrante e uma segunda que brilha ao se ater ao material-fonte, mas que falha em ser um todo coeso e em corresponder plenamente ao legado da história em quadrinhos.

--------------------------------------------------
🤔 Você: Vale a pena eu assistir esse filme?


**🤖 Gemini:** Com base nas críticas analisadas, o filme "Batman: A Piada Mortal" gerou grande decepção, com apenas 1 de 5 críticas positivas nesta amostra.

**Pontos negativos:**
*   A maioria dos espectadores aponta falta de coesão e material extra (especialmente o primeiro ato focado em Barbara Gordon) como desnecessário e mal-executado, prejudicando a fluidez e fidelidade à graphic novel.
*   Muitos sentiram que o filme se tornou duas histórias distintas.

**Pontos positivos:**
*   Há um consenso quase absoluto sobre a excelência das performances vocais, especialmente Mark Hamill como Coringa e Kevin Conroy como Batman.
*   A parte do filme que se dedica à adaptação direta da graphic novel original é geralmente vista como bem-sucedida e "fantástica".

Em resumo, o filme é percebido como uma obra de duas metades: uma primeira frustrante e desconexa, e uma segunda que brilha ao se ater ao material-fonte e às atuações dos dubladores. Se você valoriza as atuações de voz e a adaptação fiel da graphic novel na segunda metade, pode valer a pena, mas esteja ciente da recepção negativa e da falta de coesão geral.

--------------------------------------------------
🤔 Você: Sair
👋 Chat encerrado.
